# Step 5.1: Machine Learning - A first step

We are now set to perform some Machine Learning on the Asteroid Spectra Data! We keep it simple though: The multiclass clssificaiton problem of the Main Group classes:

- C
- S
- X
- Other

... shall be transformed into a binary problem. E.g.: X (1) and Not-X (0). In this script we use a Support Vector Machine (SVM) algorithm to perform some classification tasks

In [68]:
# Import standard libraries
import os

# Import installed libraries
import numpy as np
import pandas as pd
import sklearn

In [69]:
# Let's mount the Google Drive, where we store files and models (if applicable, otherwise contains working locally)
try:
    from google.colab import drive
    drive.mount('/gdrive')
    core_path = "/gdrive/MyDrive/Colab/asteroid_taxonomy/"
except ModuleNotFoundError:
    core_path = ""

In [70]:
# Load the level 2 asteroid data
asteroids_df = pd.read_pickle(os.path.join(core_path, "data/lvl2/", "asteroids.pkl"))

In [71]:
# Now we add a binary classification schema, where we distinguish between e.g., X and non-X classes
asteroids_df.loc[:, "Class"] = asteroids_df["Main_Group"].apply(lambda x: 1 if x=="X" else 0)

In [72]:
# Allocate the spectra to one array and the classes to another one
asteroids_X = np.array([k["Reflectance_norm550nm"].tolist() for k in asteroids_df["SpectrumDF"]])
asteroids_y = np.array(asteroids_df["Class"].to_list())

In [73]:
# In this example we create a single test-training split with a ratio of 0.8 / 0.2
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in sss.split(asteroids_X, asteroids_y):
    X_train, X_test = asteroids_X[train_index], asteroids_X[test_index]
    y_train, y_test = asteroids_y[train_index], asteroids_y[test_index]

In [100]:
# Let's take a look whether the unbalanced ratio has been preserved
print(f"Ration of positive training classes: {round(sum(y_train) / len(X_train), 2)}")
print(f"Ration of positive test classes: {round(sum(y_test) / len(X_test), 2)}")

Ration of positive training classes: 0.18
Ration of positive test classes: 0.18


## Unbalaned Datasets
The inverse of the ratio is approximately 5. We need this, the weight our unbalanced training set during the ML fitting process

In [101]:
# Compute class weightning
positive_class_weight = int(1.0 / (sum(y_train) / len(X_train)))
print(f"Positive Class weightning: {positive_class_weight}")

Positive Class weightning: 5


## Scaling

Scaling the training and test data should be done AFTER the splitting to prevent data leakage from the test data to the training data!

In [79]:
# Import the preprocessing module
from sklearn import preprocessing

# Instantiate the StandardScaler (mean 0, standard deviation 1) and use the training data to fit the scaler
scaler = preprocessing.StandardScaler().fit(X_train)

# Transform now the training data
X_train_scaled = scaler.transform(X_train)

## The training

Please note: This training is rather simple; it does not use multiple training / test splits, performs no logging and performs no GridSearch to find the best set of parameters!

In [91]:
# Import the SVM class
from sklearn import svm

# Call the SVM class, use an RBF kernel and apply the class weightning. That's it!
wclf = svm.SVC(kernel='rbf', class_weight={1: positive_class_weight}, C=100)

# Perform the training
wclf.fit(X_train_scaled, y_train)

In [91]:
# Scale the testing data ...
X_test_scaled = scaler.transform(X_test)

# ... and perform a predicition
y_test_pred = wclf.predict(X_test_scaled)

## Metrics - a first look

<i>What kind of metrics are useful to determine the performance of a classifier?</i>

The answer to this questions depends on one's expectations on a classifiers behaviour: Is the "overall" performance important? Does one want to prevent false positives? Or false negatives? Is the right positive result the most important metric?

Fictional project scope #1: Let's assume that our final model shall support scientists later as a tool to find X Class spectra in a vast dataset. Further, let's assume that the scientist's task is to find as many X Class spectra as possible. Consequently the scientists does not want to have falsely classified X spectra. We take a look first at the confusion matrix:

In [91]:
# Import the confusion matrix and perform the computation
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(y_test, y_test_pred)

print(conf_mat)

# The order of the confusion matrix is:
#     - true negative (top left, tn)
#     - false positive (top right, fp)
#     - false negative (bottom left, fn)
#     - true positive (bottom right, tp)
tn, fp, fn, tp = conf_mat.ravel()

[[217   4]
 [  2  45]]


## Metrics - what matters

Fictional project scope #2: The number of false negatives should be as small as possible (the scientist does not want to lose preciouse spectra!). Further, the number of false positive might be higher. The scientists will sort them out later on. A less strict requirement for the so called False Discovery Rate enables the Machine Learning Engineer to find a solution quicker in a broader solution space

In [104]:
# Recall: ratio of correctly classified X Class spectra, considering the false negatives (recall = tp / (tp + fn))
recall_score = round(sklearn.metrics.recall_score(y_test, y_test_pred), 3)
print(f"Recall Score: {recall_score}")

# Precision: ratio of correctly classified X Class spectra, considering the false positives (precision = tp / (tp + fp))
precision_score = round(sklearn.metrics.precision_score(y_test, y_test_pred), 3)
print(f"Precision Score: {precision_score}")

# A combined score
f1_score = round(sklearn.metrics.f1_score(y_test, y_test_pred), 3)
print(f"F1 Score: {precision_score}")

Recall Score: 0.957
Precision Score: 0.918
F1 Score: 0.918
